In [1]:
from typing import List, Dict, Any

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

try:
    
    from langchain_core.output_parsers import StructuredOutputParser, ResponseSchema
except ImportError:
    
    from langchain.output_parsers import StructuredOutputParser, ResponseSchema

def build_llm(model_name: str = "gpt-4o-mini", temperature: float = 0.0):
    """필요시 모델명 변경 가능 (예: gpt-4o)."""
    return ChatOpenAI(model=model_name, temperature=temperature)


In [2]:
def normalize_trip_result(d: Dict[str, Any]) -> Dict[str, Any]:
    """후처리:
    - 활동이 문자열이면 리스트로 바꾸기
    - rating이 '4점', '4/5' 같은 표현이면 숫자/문자 '4'로 정규화
    - 필드 공백 제거
    """
    if not isinstance(d, dict):
        return d

    out = dict(d)

   
    for k in ["destination", "duration", "budget", "rating"]:
        if k in out and isinstance(out[k], str):
            out[k] = out[k].strip()

    
    acts = out.get("activities")
    if isinstance(acts, str):
       
        parts = [p.strip() for p in acts.replace("·", ",").replace("|", ",").split(",") if p.strip()]
        out["activities"] = parts
    elif acts is None:
        out["activities"] = []
    elif isinstance(acts, list):
        out["activities"] = [str(x).strip() for x in acts]

    
    r = out.get("rating")
    if isinstance(r, str):
        import re
        m = re.search(r"([1-5])", r)
        if m:
            out["rating"] = m.group(1)

    return out


def parse_trip_text(text: str, llm=None) -> Dict[str, Any]:
    """
    여행 후기/계획 텍스트에서 다음 필드를 구조화:
    - destination (여행지)
    - duration (기간)
    - budget (예산)
    - rating (추천도 1~5)
    - activities (주요 활동 리스트)
    """
    llm = llm or build_llm()

    schemas = [
        ResponseSchema(name="destination", description="여행지 이름(도시/지역)"),
        ResponseSchema(name="duration", description="기간(예: 2박 3일, 3일, 주말 등)"),
        ResponseSchema(name="budget", description="예산(예: 30만원, 50만 원 등)"),
        ResponseSchema(name="rating", description="추천도(1~5) 문자열 또는 숫자 허용"),
        ResponseSchema(name="activities", description="주요 활동 리스트 (문장이라면 항목으로 분해)"),
    ]
    parser = StructuredOutputParser.from_response_schemas(schemas)
    fmt = parser.get_format_instructions()

    prompt = PromptTemplate(
        template=(
            """
            다음 여행 후기/계획 텍스트에서 핵심 정보를 추출하라.
            필수 필드: destination, duration, budget, rating, activities(list)
            - activities는 가능한 한 자연스러운 항목 리스트로 분리
            {format_instructions}

            텍스트: {text}
            """
        ),
        input_variables=["text"],
        partial_variables={"format_instructions": fmt},
    )

    chain = prompt | llm | parser
    raw = chain.invoke({"text": text})
    return normalize_trip_result(raw)


In [3]:
trip_sample = (
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 "
    "해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. "
    "정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
)

parsed = parse_trip_text(trip_sample)
parsed


{'destination': '부산',
 'duration': '2박 3일',
 'budget': '30만원',
 'rating': '4',
 'activities': ['해운대에서 바다구경', '자갈치시장에서 회 먹기', '감천문화마을 구경']}